In [1]:
import numpy
%matplotlib inline
from matplotlib import pyplot
from matplotlib import rcParams
rcParams['font.family']='serif'
rcParams['font.size']=16

In [2]:
def computef1(u1,u2,u3):
    return u2

In [3]:
def computef2(u1,u2,u3):
    return u2**2/u1+(gamma-1)*(u3-.5*u2**2/u1)

In [4]:
def computef3(u1,u2,u3):
    return (u3+(gamma-1)*(u3-.5*u2**2/u1))*u2/u1

In [5]:
def rho_initial(nx):
    rho=numpy.ones(nx)
    rho[(nx-1)/2.:]=.125
    return rho

In [6]:
def v_initial(nx):
    v=numpy.zeros(nx)
    return v

In [7]:
def p_initial(nx):
    p=numpy.ones(nx)*100000.0
    p[(nx-1)/2.:]=10000.0
    return p

In [8]:
def predictor(u,f,dx,dt):
    return .5*(u[1:]+u[:-1])-dt/(2*dx)*(f[1:]-f[:-1])

In [9]:
def corrector(u,f_med,dx,dt):
    return u[1:]-dt/dx*(f_med[1:]-f_med[:-1])

In [10]:
nx=81
dx=.25
dt=.0002
gamma=1.4
t=.01
nt=int(t/dt)+1
x=numpy.linspace(-10,10,nx)

In [11]:
rho=rho_initial(nx)
p=p_initial(nx)
v=v_initial(nx)
et=numpy.zeros(nx)
et=p/((gamma-1)*rho)+.5*v**2
u1=rho
u2=rho*v
u3=rho*et

C:\Users\yujia\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  app.launch_new_instance()


In [12]:
def richtmyer(u1,u2,u3,nt,nx,dt,dx):
    u1_n=numpy.zeros((nt,nx))
    u2_n=numpy.zeros((nt,nx))
    u3_n=numpy.zeros((nt,nx))
    u1_med=numpy.empty_like(u1)
    u2_med=numpy.empty_like(u2)
    u3_med=numpy.empty_like(u3)
    u1_med=u1.copy()
    u2_med=u2.copy()
    u3_med=u3.copy()
    u1_n[:,:]=u1.copy()
    u2_n[:,:]=u2.copy()
    u3_n[:,:]=u3.copy()
    for n in range(1,nt):
        f1=computef1(u1,u2,u3)
        f2=computef2(u1,u2,u3)
        f3=computef3(u1,u2,u3)
        u1_med[:-1]=predictor(u1,f1,dx,dt)
        u2_med[:-1]=predictor(u2,f2,dx,dt)
        u3_med[:-1]=predictor(u3,f3,dx,dt)
        f1_med=computef1(u1_med,u2_med,u3_med)
        f2_med=computef2(u1_med,u2_med,u3_med)
        f3_med=computef3(u1_med,u2_med,u3_med)
        u1_n[n,1:]=corrector(u1,f1_med,dx,dt)
        u2_n[n,1:]=corrector(u2,f2_med,dx,dt)
        u3_n[n,1:]=corrector(u3,f3_med,dx,dt)
        u1=u1_n[n].copy()
        u2=u2_n[n].copy()
        u3=u3_n[n].copy()
        u=numpy.array([u1_n,u2_n,u3_n])
    return u

In [13]:
un=numpy.empty((3,nt,nx))
un=richtmyer(u1,u2,u3,nt,nx,dt,dx)

In [14]:
u1_t=un[0,int(t/dt),int((2.5+10)/dx)]
u2_t=un[1,int(t/dt),int((2.5+10)/dx)]
u3_t=un[2,int(t/dt),int((2.5+10)/dx)]
rho_t=u1_t
v_t=u2_t/u1_t
et_t=u3_t/u1_t
p_t=(et_t-.5*v_t**2)*rho_t*(gamma-1)
print('The velocity at x=2.5 m is {:.2f}.'.format(v_t))
print('The pressure at x=2.5 m is {:.2f}.'.format(p_t))
print('The density at x=2.5 m is {:.2f}.'.format(rho_t))

The velocity at x=2.5 m is 292.61.
The pressure at x=2.5 m is 30250.89.
The density at x=2.5 m is 0.37.
